In [1]:
# Importando bilbiotecas
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

In [2]:
# Importando os arquivos
df = pd.read_csv('/datasets/users_behavior_upd.csv')
df

,calls,minutes,messages,mb_used,is_ultimate
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [3]:
# verificando o df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   calls        3214 non-null   float64
 1   minutes      3214 non-null   float64
 2   messages     3214 non-null   float64
 3   mb_used      3214 non-null   float64
 4   is_ultimate  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
# Alterando o tipo de dado da coluna 'is_ultimate' para inteiro

df['is_ultimate'] = df['is_ultimate'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   calls        3214 non-null   float64
 1   minutes      3214 non-null   float64
 2   messages     3214 non-null   float64
 3   mb_used      3214 non-null   float64
 4   is_ultimate  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
# Verificando duplicatas
df.duplicated().sum()

0

In [6]:
# Verificando ausentes
df.isna().sum()

calls          0
minutes        0
messages       0
mb_used        0
is_ultimate    0
dtype: int64

In [7]:
# Olhando o DF mais a fundo
df.describe()

,calls,minutes,messages,mb_used,is_ultimate
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


É possível perceber que o DF está completo, não existe dados faltantes e nem duplicados. Passei a coluna is_ultimate para valores inteiros. E não preciso fazer mais nenhuma alteração no DF. Podemos prosseguir.

## Definindo os conjuntos de teste

In [8]:
# As características são todas as informações sobre os planos (calls, minutes, messages e mb_used)
features = df.drop(['is_ultimate'], axis=1)

# O resultado que queremos é se o cliente se enquadra no plano Smart(0) ou Ultra(1) 
target = df['is_ultimate']

In [9]:
# Separando os dados para treino e teste, colocando 80% em teste e o resto em treinamento
features_train, features_test, target_train, target_test = train_test_split(
    features,target , test_size=0.2, random_state=12345)

# Separando os dados para treino e validação, colocando 25% em teste e o resto em validação
features_test, features_valid, target_test, target_valid = train_test_split(
    features_test, target_test , test_size=0.75, random_state=12345)

In [10]:
len(features_train)

2571

In [11]:
len(features_valid)

483

In [12]:
len(features_test)

160

## Árvore de Decisão

In [13]:
# Alterando o hiperparâmetro max_depth para procurar qual oferece a maior acurácia
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state= 1234567, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    predictions_test = model.predict(features_test)
    print('max_depth =', depth, ': ', end='')
    print('Validação:',accuracy_score(target_valid, predictions_valid))
    print('Teste:',accuracy_score(target_test, predictions_test))

max_depth = 1 : Validação: 0.7432712215320911
Teste: 0.7625
max_depth = 2 : Validação: 0.7784679089026915
Teste: 0.7875
max_depth = 3 : Validação: 0.7846790890269151
Teste: 0.78125
max_depth = 4 : Validação: 0.7784679089026915
Teste: 0.78125
max_depth = 5 : Validação: 0.7846790890269151
Teste: 0.775
max_depth = 6 : Validação: 0.7888198757763976
Teste: 0.76875
max_depth = 7 : Validação: 0.782608695652174
Teste: 0.76875
max_depth = 8 : Validação: 0.7743271221532091
Teste: 0.7625
max_depth = 9 : Validação: 0.7867494824016563
Teste: 0.75625


Fui aumentando o hiperparâmetro max_depth, até o max_depth = 30, para procurar qual a melhor acurácia que conseguia encontrar , e foi a profundidade de 2, por isso redefini o código para 10, que deixa ele mais limpo. 

Além disso, alterei o hiperparâmetro random_state para vários valores, e o melhor que encontrei foi o que utlizei no código

## Floresta Aleatória

In [14]:
# Alterando o hiperparâmetro n_estimators para verificar qual árvore performa melhor
best_score = 0
best_est = 0
for est in range(1,11):
    model = RandomForestClassifier(random_state=1234567, n_estimators= est)
    model.fit(features_train, target_train) 
    score = model.score(features_valid, target_valid) 
    if score > best_score:
        best_score = score 
        best_est = est
print("A acurácia do melhor modelo no conjunto de validação (n_estimators = {}): {}".format(best_est, best_score))

A acurácia do melhor modelo no conjunto de validação (n_estimators = 10): 0.8012422360248447


In [15]:
model = RandomForestClassifier(random_state = 1234567, n_estimators = 10)
model.fit(features_train, target_train)

y_pred = model.predict(features_test)
accuracy = accuracy_score(target_test, y_pred)

print("A acurácia do melhor modelo no conjunto de teste (n_estimators = {}): {}".format(best_est, accuracy))

A acurácia do melhor modelo no conjunto de teste (n_estimators = 10): 0.75625


## Regressão Logística

In [16]:
model = LogisticRegression(random_state=1234567, solver='liblinear')
model.fit(features_train,target_train) 
score_train = model.score(features_train, target_train)
score_valid = model.score(features_valid, target_valid)
score_test = model.score(features_test, target_test)

In [17]:

print("Acurácia do modelo de regressão logística no conjunto de treinamento:", score_train)
print("Acurácia do modelo de regressão logística no conjunto de validação:", score_valid)
print("Acurácia do modelo de regressão logística no conjunto de teste:", score_test)

Acurácia do modelo de regressão logística no conjunto de treinamento: 0.7016725009723843
Acurácia do modelo de regressão logística no conjunto de validação: 0.7080745341614907
Acurácia do modelo de regressão logística no conjunto de teste: 0.6875


## Conclusão 

Pelos modelos que foram treinados, o que apresentou a melhor acurácia foi a Árvore de Decisão, com acurácia de 78,75%. Por isso acredito que é o melhor modelo que podemos para esse caso.